In [238]:
# Import packages
import pandas as pd
from sqlalchemy import create_engine
import json
import os

pd.set_option('display.max_rows', 200)

In [239]:
%load_ext dotenv
%dotenv ../.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [240]:
con = create_engine(os.getenv('EDM_DATA'))

In [241]:
# Build funtion to get the domains associated with a specific field
def get_domain(field, domain_path):
    with open(domain_path, "r") as domain:
        domain = json.load(domain)
    records = [i for i in domain['fields'] if i['field'] == field]
    if len(records) == 0: 
        print(f'cannot find field: {field} in domain table')
    else: 
        return records[0]['domain']

In [242]:
# Example
proxcode_domain = get_domain('proxcode', 'pluto_domains.json')
print(proxcode_domain)

{'0': 'Not Available', '1': 'Detached', '2': 'Semi-Attached', '3': 'Attached'}


In [243]:
# Example
proxcode_domain.keys()

dict_keys(['0', '1', '2', '3'])

In [244]:
# Specify latest PLUTO version
pluto = 'dcp_pluto."19v2_w_corrections"'
# Specify previous PLUTO version or the one you want to compare to
plutoprev = 'dcp_pluto."19v1"'

### Value check for the thirteen fields with expected domain values

##### includes summary statistics of unique value comparisions for the two PLUTO version, and lists of values that do not agree w/ domain list

### Zoning districts

In [245]:
sql = f'''
    WITH newcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {pluto})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist),
    prevcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {plutoprev})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist)
    SELECT a.zonedist, b.zonedist as zonedistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.zonedist = b.zonedist
    ORDER BY diff DESC;
'''

In [246]:
df = pd.read_sql(sql=sql, con=con)

In [247]:
df[df['diff'] != 0]

,zonedist,zonedistprev,countnew,countprev,diff
0,None,None,NaN,2560881.0,NaN
1,None,None,2561097.0,NaN,NaN
2,C1-9A,C1-9A,137.0,134.0,3.0
141,R3-2,R3-2,63414.0,63415.0,-1.0
142,R3X,R3X,51677.0,51678.0,-1.0
143,R7B,R7B,3684.0,3685.0,-1.0
144,R5D,R5D,5514.0,5515.0,-1.0
145,R4,R4,75974.0,75975.0,-1.0
146,C6-4X,C6-4X,88.0,89.0,-1.0
147,R3-1,R3-1,51469.0,51470.0,-1.0


### Commercial overlay

In [248]:
sql = f'''
    WITH newcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {pluto}
    UNION ALL
    SELECT overlay2 as overlay FROM {pluto}
    )
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay),
    prevcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {plutoprev}
    UNION ALL
    SELECT overlay2 as overlay FROM {plutoprev})
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay)
    SELECT a.overlay, b.overlay as overlayprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.overlay = b.overlay
    ORDER BY a.overlay;
'''

In [249]:
df = pd.read_sql(sql=sql, con=con)

In [250]:
df

,overlay,overlayprev,countnew,countprev,diff
0,C1-1,C1-1,1587.0,1587.0,0.0
1,C1-2,C1-2,11031.0,11031.0,0.0
2,C1-3,C1-3,13790.0,13790.0,0.0
3,C1-4,C1-4,8097.0,8097.0,0.0
4,C1-5,C1-5,2541.0,2547.0,-6.0
5,C2-1,C2-1,1684.0,1684.0,0.0
6,C2-2,C2-2,6023.0,6023.0,0.0
7,C2-3,C2-3,11949.0,11952.0,-3.0
8,C2-4,C2-4,16721.0,16727.0,-6.0
9,C2-5,C2-5,1523.0,1523.0,0.0


### Special purpose district

In [251]:
sql = f'''
    WITH newcount as (
    WITH sdall as (
    SELECT spdist1 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist2 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist3 as spdist FROM {pluto}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist),
    prevcount as (
     WITH sdall as (
    SELECT spdist1 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist2 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist3 as spdist FROM {plutoprev}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist)
    SELECT a.spdist, b.spdist as spdistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.spdist = b.spdist
    ORDER BY diff DESC;
'''

In [252]:
df = pd.read_sql(sql=sql, con=con)

In [253]:
df[df['diff'] != 0]

,spdist,spdistprev,countnew,countprev,diff
0,None,None,2476702.0,NaN,NaN
1,None,None,NaN,2476583.0,NaN
2,MiD,MiD,1486.0,1485.0,1.0
76,OP,OP,13442.0,13443.0,-1.0
77,EC-6,EC-6,409.0,410.0,-1.0
78,HRP,HRP,5.0,6.0,-1.0
79,HS,HS,6164.0,6165.0,-1.0
80,EC-5,EC-5,558.0,559.0,-1.0
81,MP,MP,370.0,373.0,-3.0
82,DB,DB,932.0,936.0,-4.0


In [254]:
#Check spdist domain
unique_spdist = df.spdist.tolist()
spdist_domain = list(get_domain('spdist1', 'pluto_domains.json').keys())

In [255]:
# in domain not in pluto:
[i for i in spdist_domain if i not in unique_spdist]

['MX-3']

In [256]:
# in pluto not in domain:
[i for i in unique_spdist if i not in spdist_domain]

[None, None, 'MX-16/EC-5', 'BSC', 'SW', 'MX-20', 'MX-16/EC-6']

### Limited height

In [257]:
sql = f'''
    WITH newcount as (
    SELECT ltdheight, count(*)
    FROM {pluto}
    GROUP BY ltdheight),
    prevcount as (
    SELECT ltdheight, count(*)
    FROM {plutoprev}
    GROUP BY ltdheight)
    SELECT a.ltdheight, b.ltdheight as ltdheightprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ltdheight = b.ltdheight
    ORDER BY a.ltdheight;
'''

In [258]:
df = pd.read_sql(sql=sql, con=con)

In [259]:
df

,ltdheight,ltdheightprev,countnew,countprev,diff
0,LH-1,LH-1,2239.0,2239.0,0.0
1,LH-1A,LH-1A,797.0,797.0,0.0
2,None,None,856459.0,NaN,NaN
3,None,None,NaN,856423.0,NaN


In [260]:
#Check ltdheight domain
unique_ltdheight = df.ltdheight.tolist()
ltdheight_domain = list(get_domain('ltdheight', 'pluto_domains.json').keys())

In [261]:
# in domain not in pluto:
[i for i in ltdheight_domain if i not in unique_ltdheight]

['LH-2', 'LH-3']

In [262]:
# in pluto not in domain:
[i for i in unique_ltdheight if i not in ltdheight_domain]

[None, None]

### Building class

In [263]:
sql = f'''
    WITH newcount as (
    SELECT bldgclass, count(*)
    FROM {pluto}
    GROUP BY bldgclass),
    prevcount as (
    SELECT bldgclass, count(*)
    FROM {plutoprev}
    GROUP BY bldgclass)
    SELECT a.bldgclass, b.bldgclass as bldgclassprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bldgclass = b.bldgclass
    ORDER BY diff DESC;
'''

In [264]:
df = pd.read_sql(sql=sql, con=con)

In [265]:
df[df['diff'] != 0]

,bldgclass,bldgclassprev,countnew,countprev,diff
0,None,None,509.0,NaN,NaN
1,None,None,NaN,473.0,NaN
2,Z7,Z7,167.0,109.0,58.0
204,D7,D7,2118.0,2119.0,-1.0
205,D3,D3,1306.0,1307.0,-1.0
206,U0,U0,85.0,86.0,-1.0
207,Q7,Q7,27.0,28.0,-1.0
208,A9,A9,22752.0,22753.0,-1.0
209,Q9,Q9,284.0,285.0,-1.0
210,V9,V9,501.0,502.0,-1.0


In [266]:
#Check bldgclass domain
unique_bldgclass = df.bldgclass.tolist()
bldgclass_domain = list(get_domain('bldgclass', 'pluto_domains.json').keys())

In [267]:
# in domain not in pluto:
[i for i in bldgclass_domain if i not in unique_bldgclass]

['A',
 'B',
 'C',
 'D',
 'E',
 'E3',
 'E4',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'R7',
 'S',
 'T',
 'U',
 'U3',
 'V',
 'W',
 'Y',
 'Z',
 'Z6']

In [268]:
# in pluto not in domain:
[i for i in unique_bldgclass if i not in bldgclass_domain]

[None, None]

### Land use

In [269]:
sql = f'''
    WITH newcount as (
    SELECT landuse, count(*)
    FROM {pluto}
    GROUP BY landuse),
    prevcount as (
    SELECT landuse, count(*)
    FROM {plutoprev}
    GROUP BY landuse)
    SELECT a.landuse, b.landuse as landuseprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.landuse = b.landuse
    ORDER BY a.landuse;
'''

In [270]:
df = pd.read_sql(sql=sql, con=con)

In [271]:
df[df['diff'] != 0]

,landuse,landuseprev,countnew,countprev,diff
0,01,01,565355.0,565356.0,-1.0
2,03,03,12845.0,12846.0,-1.0
3,04,04,54507.0,54508.0,-1.0
6,07,07,6308.0,6335.0,-27.0
8,09,09,4011.0,4017.0,-6.0
10,11,11,27693.0,27715.0,-22.0
11,None,None,3018.0,NaN,NaN
12,None,None,NaN,2924.0,NaN


In [272]:
#Check landuse domain
unique_landuse = df.landuse.tolist()
landuse_domain = list(get_domain('landuse', 'pluto_domains.json').keys())

In [273]:
# in domain not in pluto:
[i for i in landuse_domain if i not in unique_landuse]

[]

In [274]:
# in pluto not in domain:
[i for i in unique_landuse if i not in landuse_domain]

[None, None]

### Owner type

In [275]:
sql = f'''
    WITH newcount as (
    SELECT ownertype, count(*)
    FROM {pluto}
    GROUP BY ownertype),
    prevcount as (
    SELECT ownertype, count(*)
    FROM {plutoprev}
    GROUP BY ownertype)
    SELECT a.ownertype, b.ownertype as ownertypeprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ownertype = b.ownertype
    ORDER BY a.ownertype;
'''

In [276]:
df = pd.read_sql(sql=sql, con=con)

In [277]:
df

,ownertype,ownertypeprev,countnew,countprev,diff
0,C,C,11654.0,11654.0,0.0
1,M,M,80.0,80.0,0.0
2,O,O,1373.0,1373.0,0.0
3,P,P,522.0,522.0,0.0
4,X,X,17414.0,17414.0,0.0
5,None,None,NaN,828416.0,NaN
6,None,None,828452.0,NaN,NaN


In [278]:
#Check ownertype domain
unique_ownertype = df.ownertype.tolist()
ownertype_domain = list(get_domain('ownertype', 'pluto_domains.json').keys())

In [279]:
# in domain not in pluto:
[i for i in ownertype_domain if i not in unique_ownertype]

['NaN']

In [280]:
# in pluto not in domain:
[i for i in unique_ownertype if i not in ownertype_domain]

[None, None]

### Area source

In [281]:
sql = f'''
    WITH newcount as (
    SELECT areasource, count(*)
    FROM {pluto}
    GROUP BY areasource),
    prevcount as (
    SELECT areasource, count(*)
    FROM {plutoprev}
    GROUP BY areasource)
    SELECT a.areasource, b.areasource as areasourceprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.areasource = b.areasource
    ORDER BY a.areasource;
'''

In [282]:
df = pd.read_sql(sql=sql, con=con)

In [283]:
df

,areasource,areasourceprev,countnew,countprev,diff
0,0,0,139.0,136.0,3.0
1,2,2,813263.0,813263.0,0.0
2,4,4,179.0,182.0,-3.0
3,5,5,63.0,63.0,0.0
4,7,7,45342.0,45342.0,0.0
5,None,None,509.0,NaN,NaN
6,None,None,NaN,473.0,NaN


In [284]:
#Check areasource domain
unique_areasource = df.areasource.tolist()
areasource_domain = list(get_domain('areasource', 'pluto_domains.json').keys())

In [285]:
# in domain not in pluto:
[i for i in areasource_domain if i not in unique_areasource]

[]

In [286]:
# in pluto not in domain:
[i for i in unique_areasource if i not in areasource_domain]

[None, None]

### Extension

In [287]:
sql = f'''
    WITH newcount as (
    SELECT ext, count(*)
    FROM {pluto}
    GROUP BY ext),
    prevcount as (
    SELECT ext, count(*)
    FROM {plutoprev}
    GROUP BY ext)
    SELECT a.ext, b.ext as extprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ext = b.ext
    ORDER BY a.ext;
'''

In [288]:
df = pd.read_sql(sql=sql, con=con)

In [289]:
df

,ext,extprev,countnew,countprev,diff
0,E,E,48335.0,48335.0,0.0
1,EG,EG,36709.0,36709.0,0.0
2,G,G,264413.0,264413.0,0.0
3,N,N,451333.0,451333.0,0.0
4,None,None,58705.0,NaN,NaN
5,None,None,NaN,58669.0,NaN


In [290]:
#Check extension domain
unique_extension = df.ext.tolist()
extension_domain = list(get_domain('ext', 'pluto_domains.json').keys())

In [291]:
# in domain not in pluto:
[i for i in extension_domain if i not in unique_extension]

[]

In [292]:
# in pluto not in domain:
[i for i in unique_extension if i not in extension_domain]

['N', None, None]

### Proxy code

In [293]:
sql = f'''
    WITH newcount as (
    SELECT proxcode, count(*)
    FROM {pluto}
    GROUP BY proxcode),
    prevcount as (
    SELECT proxcode, count(*)
    FROM {plutoprev}
    GROUP BY proxcode)
    SELECT a.proxcode, b.proxcode as proxcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.proxcode = b.proxcode
    ORDER BY a.proxcode;
'''

In [294]:
df = pd.read_sql(sql=sql, con=con)

In [295]:
df

,proxcode,proxcodeprev,countnew,countprev,diff
0,0,0,144832.0,144832.0,0.0
1,1,1,313731.0,313731.0,0.0
2,2,2,199995.0,199995.0,0.0
3,3,3,200428.0,200428.0,0.0
4,None,None,509.0,NaN,NaN
5,None,None,NaN,473.0,NaN


In [296]:
#Check proxcode domain
unique_proxcode = df.proxcode.tolist()
proxcode_domain = list(get_domain('proxcode', 'pluto_domains.json').keys())

In [297]:
# in domain not in pluto:
[i for i in proxcode_domain if i not in unique_proxcode]

[]

In [298]:
# in pluto not in domain:
[i for i in unique_proxcode if i not in proxcode_domain]

[None, None]

### Lot type

In [299]:
sql = f'''
    WITH newcount as (
    SELECT lottype, count(*)
    FROM {pluto}
    GROUP BY lottype),
    prevcount as (
    SELECT lottype, count(*)
    FROM {plutoprev}
    GROUP BY lottype)
    SELECT a.lottype, b.lottype as lottypeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.lottype = b.lottype
    ORDER BY a.lottype;
'''

In [300]:
df = pd.read_sql(sql=sql, con=con)

In [301]:
df

,lottype,lottypeprev,countnew,countprev,diff
0,0,0,8534.0,8534.0,0.0
1,1,1,1454.0,1456.0,-2.0
2,2,2,1823.0,1827.0,-4.0
3,3,3,100674.0,100674.0,0.0
4,4,4,3443.0,3438.0,5.0
5,5,5,738045.0,738045.0,0.0
6,6,6,2228.0,2228.0,0.0
7,7,7,64.0,64.0,0.0
8,8,8,2531.0,2530.0,1.0
9,9,9,190.0,190.0,0.0


In [302]:
#Check lottype domain
unique_lottype = df.lottype.tolist()
lottype_domain = list(get_domain('lottype', 'pluto_domains.json').keys())

In [303]:
# in domain not in pluto:
[i for i in lottype_domain if i not in unique_lottype]

[]

In [304]:
# in pluto not in domain:
[i for i in unique_lottype if i not in lottype_domain]

[None, None]

### Basement code

In [305]:
sql = f'''
    WITH newcount as (
    SELECT bsmtcode, count(*)
    FROM {pluto}
    GROUP BY bsmtcode),
    prevcount as (
    SELECT bsmtcode, count(*)
    FROM {plutoprev}
    GROUP BY bsmtcode)
    SELECT a.bsmtcode, b.bsmtcode as bsmtcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bsmtcode = b.bsmtcode
    ORDER BY a.bsmtcode;
'''

In [306]:
df = pd.read_sql(sql=sql, con=con)

In [307]:
df

,bsmtcode,bsmtcodeprev,countnew,countprev,diff
0,0,0,74507.0,74508.0,-1.0
1,1,1,153674.0,153697.0,-23.0
2,2,2,495893.0,495979.0,-86.0
3,3,3,1499.0,1498.0,1.0
4,4,4,4704.0,4705.0,-1.0
5,5,5,128709.0,128599.0,110.0
6,None,None,509.0,NaN,NaN
7,None,None,NaN,473.0,NaN


In [308]:
#Check bsmtcode domain
unique_bsmtcode = df.bsmtcode.tolist()
bsmtcode_domain = list(get_domain('bsmtcode', 'pluto_domains.json').keys())

In [309]:
# in domain not in pluto:
[i for i in bsmtcode_domain if i not in unique_bsmtcode]

[]

In [310]:
# in pluto not in domain:
[i for i in unique_bsmtcode if i not in bsmtcode_domain]

[None, None]

### PLUTO map id

In [311]:
sql = f'''
    WITH newcount as (
    SELECT plutomapid, count(*)
    FROM {pluto}
    GROUP BY plutomapid),
    prevcount as (
    SELECT plutomapid, count(*)
    FROM {plutoprev}
    GROUP BY plutomapid)
    SELECT a.plutomapid, b.plutomapid as plutomapidprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.plutomapid = b.plutomapid
    ORDER BY a.plutomapid;
'''

In [312]:
df = pd.read_sql(sql=sql, con=con)

In [313]:
df

,plutomapid,plutomapidprev,countnew,countprev,diff
0,1,1,856793,856861,-68
1,2,2,1983,1915,68
2,3,3,505,469,36
3,4,4,210,210,0
4,5,5,4,4,0


In [314]:
#Check plutomapid domain
unique_plutomapid = df.plutomapid.tolist()
plutomapid_domain = list(get_domain('plutomapid', 'pluto_domains.json').keys())

In [315]:
# in domain not in pluto:
[i for i in plutomapid_domain if i not in unique_plutomapid]

[]

In [316]:
# in pluto not in domain:
[i for i in unique_plutomapid if i not in plutomapid_domain]

[]

#### done